In [1]:
import pandas as pd
import numpy as np

In [ ]:
magnus_df = pd.read_excel(r'Germany Analytics report_Magnus_v7.xlsx',sheetname="Report")

df_m_1 = magnus_df.copy()

df_m_1 = df_m_1.iloc[3:,:].reset_index()
cols = df_m_1.iloc[:1,:].values[0]
df_m_1.columns = cols
df_m_1 = df_m_1.iloc[1:,]

df_m_1.columns = df_m_1.columns.str.strip()
df_m_1.columns = df_m_1.columns.str.replace(" ","")

df_m_1['ContractStart']= pd.to_datetime(df_m_1['ContractStart'],infer_datetime_format=True)
df_m_1['ContractEnd']= pd.to_datetime(df_m_1['ContractEnd'],infer_datetime_format=True)

# plantCode-Region Mapping
plant_map = pd.read_pickle('../plant_map.pkl')

df_m_1 = pd.merge(df_m_1,plant_map[['plantcode','region']],how='left',on='plantcode')

df_m_1.loc[df_m_1.plantcode =='0NFI' ,'region']= "WEST"

df_m_1.loc[df_m_1.plantcode =='0600' ,'region']= "SOUTHEAST"

sel_cols = pd.read_excel(r'../sel_cols.xlsx')
sel_cols = sel_cols.query('is_sel == 1')['colName'].tolist()

df_m_2 = df_m_1.loc[:,sel_cols]

df_m_2['CaseStatus'] = df_m_2.CaseStatus.str.replace(" ","")
df_m_2 = df_m_2.query('CaseStatus == "Releasedtopricing" & IsLatestCycle == "Yes" ')
#df_m_2 = df_m_2.drop_duplicates(subset=['plantcode','SAPOrderNo','grade','Material#','Material'], keep='last')
df_m_2 = df_m_2.drop_duplicates(subset=['SAPOrderNo','Material#','Material','grade'], keep='last')
df_m_2.loc[df_m_2.grade.isnull(),'Material'].value_counts()
df_m_2.loc[df_m_2.grade.isnull(),'grade']="ADO"
df_m_2.dropna(inplace=True)
df_m_2.to_pickle("df_m_2.pkl")

In [ ]:
sap_df_raw = pd.read_excel(r'Liftings by contract and product_V2.xlsx',sheetname="Sheet2")

sap_df_raw.to_pickle('sap_df_raw.pkl')

df_s = sap_df_raw.iloc[10:,2:]
cols = df_s.iloc[:1,:].values[0][2:].tolist()
cols.insert(0,'contract')
cols.insert(1,'Products')
df_s.columns = cols
df_s = df_s.iloc[2:,:]
df_s.fillna(0,inplace=True)
df_s_1 = pd.melt(df_s,id_vars=["contract","Products"],var_name="Date",value_name="Volume")
df_s_1['Date']= pd.to_datetime(df_s_1['Date'],infer_datetime_format=True)
df_s_1.loc[df_s_1['Volume']<0,"Volume"] =0 
df_s_1.to_pickle("new_sap_1.pkl")

df_s_1['ProductId'] = df_s_1.Products.str.replace(" ","").str[:6].astype(str)

df_s_1['contract'] = df_s_1['contract'].str.replace(" ", "")
df_m_2['SAPOrderNo'] = df_m_2['SAPOrderNo'].str.replace(" ", "")
df_s_1['ProductId'] = df_s_1['ProductId'].str.strip().astype(str)
df_m_2['Material#'] = df_m_2['Material#'].astype(str)

In [56]:
df_s_2 = pd.merge(df_s_1,df_m_2[['grade','Material#','SAPOrderNo','region','ContractVolume','SpotPrice']],
                  how='left',left_on=['contract','ProductId'],right_on=['SAPOrderNo','Material#'])
df_s_2.dropna(inplace=True)

In [139]:
df_a = pd.read_excel(r'ARM grids OMR vs average Sales Price 2019 27122019_V2.xlsx',sheetname="Taxed prices")
df_a.to_pickle('arm_grid_df.pkl')

df_a = df_a.reset_index()
df_a = df_a.iloc[10:,3:]
cols = df_a.iloc[:1,:].values[0].tolist()
df_a.columns = cols
df_a = df_a.iloc[1:,]

df_a.ffill(axis =0 ,inplace= True)
df_a_1 = pd.concat([df_a.iloc[:,0],df_a.iloc[:,16:37]],axis=1)
df_a_1['Quote date']= pd.to_datetime(df_a_1['Quote date'],infer_datetime_format=True)
df_a_2 = pd.melt(df_a_1,id_vars=["Quote date"],var_name="Product",value_name="quote")

prd_map = pd.read_excel(r'../all_data_set.xlsx',sheetname="product_mapping")
df_a_3 = pd.merge(df_a_2,prd_map,how='left',left_on='Product',right_on='product')

df_a_3.dropna(inplace=True)

In [152]:
df_s_3 = pd.merge(df_s_2,df_a_3[['Quote date', 'quote', 'region', 'Grade ']],
                  how='left',left_on=['Date','grade','region'],right_on=['Quote date','Grade ','region'])

In [174]:
df_s_3.to_pickle('df_s_3.pkl')

In [157]:
df_s_3.to_excel(r'prepared_data_viz.xlsx')

In [159]:
df_s_3.loc[df_s_3.Volume < 0 , 'Volume'] = 0
df_s_3['SpotPrice_adj'] = df_s_3['SpotPrice']*0.1
df_s_3['contractValue'] = df_s_3['SpotPrice_adj']*df_s_3['ContractVolume']
df_s_3['volByquote'] = df_s_3['Volume']*df_s_3['quote']

df_s_3['SpotPrice'] =  df_s_3['SpotPrice'].astype('float64')
df_s_3['contractValue'] =  df_s_3['contractValue'].astype('float64')
df_s_3['SpotPrice_adj'] =  df_s_3['SpotPrice_adj'].astype('float64')
df_s_3['volByquote'] =  df_s_3['volByquote'].astype('float64')

df_s_3['contractValue_adj'] = df_s_3['SpotPrice_adj']*df_s_3['Volume']

df_s_3['ContractVolume'] =  df_s_3['ContractVolume'].astype('float64')

In [169]:
df_s_4 = df_s_3.groupby('contract').agg({
                                        'Volume' : 'sum',
                                        'ContractVolume':'mean',
                                        'contractValue' : 'mean',
                                        'contractValue_adj': 'sum',
                                        'volByquote' : 'sum'
                                   }).reset_index()

In [171]:
df_s_4['Profit'] =  df_s_4.contractValue_adj - df_s_4.volByquote
#df_s_4 = pd.merge(df_s_4,df_m_2,how='left',left_on=['contract'],right_on=['SAPOrderNo'])

In [172]:
df_s_4['Profit'].sum()

-282930.78025314194

In [173]:
df_s_4.to_excel('profit_calc_1.xlsx')

In [176]:
df_a_3 = pd.read_pickle('df_a_3.pkl')